# Text-to-SQL Lab 

**Last updated:** 2025-10-23

This notebook is designed for a hands-on lab. It starts from a working
Text-to-SQL pipeline and adds structured guidance, exercises, and safety checks.
Students can explore prompting strategies, schema linking, validation, and evaluation.

## Learning objectives
By the end of the lab, students will be able to:
- Explain the basic Text-to-SQL pipeline and its risks.
- Inspect a SQLite schema and design prompts that condition the model on schema.
- Generate and execute SQL safely (read-only; parameterized where relevant).
- Evaluate query correctness with simple heuristics and unit-style tests.
- Modularize code into files to build a small FastAPI app.

## Prerequisites
- Python 3.10+
- A local SQLite database (provided or generated in this notebook)
- An LLM provider key (e.g., OpenAI API key) available as an environment variable

> Tip: **Never** run untrusted SQL that can mutate the database. In this lab we restrict to `SELECT` queries.

## Setup

1. Create a virtual environment and install dependencies (example):
   ```bash
   python -m venv .venv && source .venv/bin/activate
   pip install -r requirements.txt
   ```

2. Set your API key (example for OpenAI) in a `.env` file:
   ```bash
   OPENAI_API_KEY="sk-..."
   ```

3. Run the first section to create / connect to the sample SQLite database.

## Lab map

1. **Data & schema**: create or connect to SQLite; inspect tables.
2. **Baseline text→SQL**: prompt LLM, generate SQL, run the query.
3. **Safety pass**: restrict to `SELECT`, block DDL/DML, and validate SQL.
4. **Prompt engineering**: add schema, few-shots, and constraints.
5. **Evaluation**: define expected outputs, compare, and log failures.
6. **Refactor**: extract functions and move into a FastAPI app.

Based on: Creating a Text to SQL App with OpenAI + FastAPI + SQLite

In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
import os
import json
from openai import OpenAI
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sqlalchemy import inspect
import pandas as pd
import sqlite3
from pathlib import Path
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import Engine

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

## 📘 Step 1 — Create & seed the demo database

**Why this matters**
- A consistent, realistic schema lets you practice schema grounding and query reasoning.
- Seeding deterministic data enables repeatable evaluation later.
- We’ll use this DB both in-notebook and later when we refactor into a FastAPI app.

**What’s provided**
- Code to (re)create `demo.db` and populate `customers`, `products`, `orders`, `order_items`, `payments`.

**Students do**
- [ ] Inspect the schema (tables, columns, types) via PRAGMA queries and helper functions.  
- [ ] Produce short **schema bullets** (1–2 lines/table) to feed into the prompt later.

**Checks**
- You can list the tables and see sensible row counts.
- `PRAGMA foreign_key_list(…)` shows expected relationships.

**Stretch**
- Add an integrity check: verify that `orders.total` equals the sum of `order_items.quantity * unit_price` for each order.
- Add a view `order_totals` and compare it to the stored `orders.total`.

In [3]:
SCHEMA_SQL = '''
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS payments;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    country TEXT,
    signup_date DATE
);

CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    category TEXT,
    price REAL
);

CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);

CREATE TABLE order_items (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    unit_price REAL,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);

CREATE TABLE payments (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    payment_date DATE,
    amount REAL,
    method TEXT,
    FOREIGN KEY (order_id) REFERENCES orders(id)
);

INSERT INTO customers (id, name, country, signup_date) VALUES
 (1,'Alice','USA','2024-01-05'),
 (2,'Bob','UK','2024-03-10'),
 (3,'Choi','KR','2024-06-22'),
 (4,'Dara','ID','2025-01-15');

INSERT INTO products (id, name, category, price) VALUES
 (1,'Laptop Pro','Electronics',1500.00),
 (2,'Noise-Canceling Headphones','Electronics',300.00),
 (3,'Standing Desk','Furniture',450.00),
 (4,'Ergonomic Chair','Furniture',250.00),
 (5,'Monitor 27"', 'Electronics',350.00);

INSERT INTO orders (id, customer_id, order_date, total) VALUES
 (1,1,'2025-02-01',1850.00),
 (2,2,'2025-02-03',600.00),
 (3,3,'2025-02-05',350.00),
 (4,1,'2025-02-07',450.00);

INSERT INTO order_items (order_id, product_id, quantity, unit_price) VALUES
 (1,1,1,1500.00),
 (1,2,1,300.00),
 (1,5,1,350.00),
 (2,3,1,450.00),
 (2,4,1,250.00),
 (3,5,1,350.00),
 (4,3,1,450.00);

INSERT INTO payments (id, order_id, payment_date, amount, method) VALUES
 (1,1,'2025-02-01',1850.00,'Credit Card'),
 (2,2,'2025-02-03',600.00,'PayPal'),
 (3,3,'2025-02-05',350.00,'Credit Card'),
 (4,4,'2025-02-07',450.00,'Bank Transfer');
'''
DB_PATH = Path("demo.db")
if DB_PATH.exists():
    DB_PATH.unlink()
with sqlite3.connect(DB_PATH) as conn:
    conn.executescript(SCHEMA_SQL)
print("Created", DB_PATH.resolve())

Created C:\git_repository\TAG-Exercise\demo.db


### 🧭 Step 1 — Schema inspection tasks

- Use:
  - `SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';`
  - `PRAGMA table_info(<table>);`
  - `PRAGMA foreign_key_list(<table>);`
- Summarize each table in 1–2 lines, e.g.  
  `orders(id PK, customer_id INTEGER, order_date DATE, total REAL; customer_id → customers.id)`
- Save your bullets in a Python list (e.g., `schema_bullets = [...]`) to reuse in prompts.
- Optional checks:
  - Count rows per table.
  - Verify that every `order_items.order_id` exists in `orders.id`.

In [14]:
# Step 1 — Schema inspection helpers (STUDENTS)

import sqlite3
from pathlib import Path

DB_PATH = Path("demo.db")

# === TODO (students) ===
# Implement the helpers below to explore the schema and produce short, prompt-ready bullets.

def list_tables(conn):
    """
    Return a list of table names (excluding SQLite internal tables).
    """
    query = """
        SELECT name
        FROM sqlite_master
        WHERE type = 'table'
          AND name NOT LIKE 'sqlite_%';
    """
    
    cursor = conn.cursor()
    cursor.execute(query)
    tables = [row[0] for row in cursor.fetchall()]
    cursor.close()
    
    return tables

def describe_table(conn, table):
    """
    Return a list of dicts: [{'name': 'col', 'type': 'TEXT', 'pk': 0, 'notnull': 0, 'dflt_value': None}, ...]
    """
    query = f"PRAGMA table_info({table});"
    
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [
        {
            'name': row[1],
            'type': row[2],
            'pk': row[5],
            'notnull': row[3],
            'dflt_value': row[4]
        }
        for row in cursor.fetchall()
    ]
    cursor.close()

    return columns

def foreign_keys(conn, table):
    """
    Return a list of foreign key dicts with 'from', 'to_table', 'to_col'.
    """
    
    # TODO: use PRAGMA foreign_key_list(table);
    query = f"PRAGMA foreign_key_list({table});"

    cursor = conn.cursor()
    cursor.execute(query)
    fks = [
        {
            'from': row[3],
            'to_table': row[2],
            'to_col': row[4]
        }
        for row in cursor.fetchall()
    ]
    cursor.close()

    return fks

    

def schema_bullets(conn):
    """
    Produce 1–2 line summaries per table to condition the LLM prompt.
    Example bullet: "orders(id PK, customer_id → customers.id, order_date DATE, total REAL)"
    """
    bullets = []
    for table in list_tables(conn):
        columns = describe_table(conn, table)
        fks = foreign_keys(conn, table)

        col_bullets = []
        for col in columns:
            col_desc = f"{col['name']} {col['type']}"
            if col['pk']:
                col_desc += " PK"
            col_bullets.append(col_desc)

        fk_bullets = []
        for fk in fks:
            fk_bullets.append(f"{fk['from']} → {fk['to_table']}.{fk['to_col']}")

        bullets.append(f"{table}({', '.join(col_bullets + fk_bullets)})")

    return bullets

# --- Checks ---
with sqlite3.connect(DB_PATH) as conn:
    print("Tables:", list_tables(conn))
    print("--------------------")
    for t in list_tables(conn):
        print(f"\n[{t}]")
        print("Columns:", describe_table(conn, t))
        print("FKs:", foreign_keys(conn, t))
    print("--------------------")
    print("\nSchema bullets:")
    for b in schema_bullets(conn):
        print("-", b)

Tables: ['customers', 'products', 'orders', 'order_items', 'payments']
--------------------

[customers]
Columns: [{'name': 'id', 'type': 'INTEGER', 'pk': 1, 'notnull': 0, 'dflt_value': None}, {'name': 'name', 'type': 'TEXT', 'pk': 0, 'notnull': 1, 'dflt_value': None}, {'name': 'country', 'type': 'TEXT', 'pk': 0, 'notnull': 0, 'dflt_value': None}, {'name': 'signup_date', 'type': 'DATE', 'pk': 0, 'notnull': 0, 'dflt_value': None}]
FKs: []

[products]
Columns: [{'name': 'id', 'type': 'INTEGER', 'pk': 1, 'notnull': 0, 'dflt_value': None}, {'name': 'name', 'type': 'TEXT', 'pk': 0, 'notnull': 1, 'dflt_value': None}, {'name': 'category', 'type': 'TEXT', 'pk': 0, 'notnull': 0, 'dflt_value': None}, {'name': 'price', 'type': 'REAL', 'pk': 0, 'notnull': 0, 'dflt_value': None}]
FKs: []

[orders]
Columns: [{'name': 'id', 'type': 'INTEGER', 'pk': 1, 'notnull': 0, 'dflt_value': None}, {'name': 'customer_id', 'type': 'INTEGER', 'pk': 0, 'notnull': 0, 'dflt_value': None}, {'name': 'order_date', 'type'

# 📘 Step 1 — Create & Seed the Demo Database

This section handles the crucial step of **Database Grounding**. We create a consistent SQLite database and establish a robust, **read-only connection**. A read-only connection is a vital **security measure** to ensure the LLM cannot execute destructive SQL commands (like `DROP TABLE` or `DELETE`).

### 1.1 Database Setup (Provided Code)

The code below performs three main actions:

1.  **Engine Creation:** It uses **SQLAlchemy** to create a database engine. We choose SQLAlchemy over the standard `sqlite3` library because it provides a uniform way to handle database URIs and connection pooling, which is essential for our final step: building a scalable FastAPI application.
2.  **Security Measures:** The connection URI includes `mode=ro&uri=true`, and we add a layer of **defense-in-depth** by executing `PRAGMA query_only = ON;` to explicitly prevent write operations.
3.  **Schema Extraction:** It uses SQLAlchemy's `inspect` to automatically generate a simple `CREATE TABLE` string (`SCHEMA_STR`). This string is the **baseline schema prompt** fed to the LLM.

In [9]:
DB_URI = f"sqlite:///file:{DB_PATH.resolve()}?mode=ro&uri=true"
engine: Engine = create_engine(DB_URI, connect_args={"uri": True})

# Defense-in-depth: block writes on the connection
with engine.connect() as conn:
    try:
        conn.exec_driver_sql("PRAGMA query_only = ON;")
    except Exception:
        pass

# Build a short schema summary to feed the LLM
insp = inspect(engine)
SCHEMA_STR = "\n".join(
    f"CREATE TABLE {t} ({', '.join(c['name'] for c in insp.get_columns(t))});"
    for t in insp.get_table_names()
)
schema = SCHEMA_STR
print(SCHEMA_STR)

CREATE TABLE customers (id, name, country, signup_date);
CREATE TABLE order_items (order_id, product_id, quantity, unit_price);
CREATE TABLE orders (id, customer_id, order_date, total);
CREATE TABLE payments (id, order_id, payment_date, amount, method);
CREATE TABLE products (id, name, category, price);


### 📝 Baseline Text-to-SQL Generation

Now that we have the schema (`SCHEMA_STR`) and a read-only database connection (`engine`), we can implement the core Text-to-SQL logic. This step involves sending the user's question and the database schema to the Large Language Model (LLM) and receiving a generated SQL query.

### The System Prompt

The **system prompt** is a critical piece of the pipeline. It defines the LLM's role, constraints, and output format. Key instructions include:

* **Role Definition:** "You convert natural-language questions into read-only SQLite SQL."
* **Safety Guardrails:** Enforcing `SELECT` queries only, blocking all data definition/manipulation language (DDL/DML) commands (`INSERT`, `UPDATE`, `DELETE`, etc.).
* **Format Constraint:** Using `response_format={"type": "json_object"}` ensures the output is consistently parseable, reducing hallucination of the surrounding text.

In [15]:
def generate_sql_openai(question: str, schema: str, model: str = None) -> str:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    _SYSTEM_PROMPT = """You convert natural-language questions into read-only SQLite SQL.
                        Rules:
                        - Output only a JSON object like: {"sql": "..."}.
                        - Use SELECT queries only. Never write INSERT/UPDATE/DELETE/DDL.
                        - Prefer clear column aliases and LIMIT if the result could be large.
                        - The target dialect is SQLite.
                    """
    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format={"type": "json_object"},
        messages=[
            {"role":"system","content": _SYSTEM_PROMPT},
            {"role":"user","content": f"schema:\n{schema}\n\nquestion: {question}"},
        ],
    )
    payload = json.loads(resp.choices[0].message.content)
    return payload["sql"]

## Executing the Generated SQL

Once the LLM successfully translates the natural language question into a `SELECT` query, the next critical step is safely executing it against the database. We use the `run_sql` function for this.

In [28]:
def run_sql(engine: Engine, sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        df = pd.read_sql_query(sql, conn)
    return df

## 🚀 Step 3 — Test the Baseline Pipeline

With the generation and execution functions ready, we can run our first complete Text-to-SQL query.

### 3.1 Initial Test (Count Query)

We'll start with a simple query that requires only one table, confirming the LLM can correctly select the table and aggregate data based on the prompt and the provided schema string (`SCHEMA_STR`).

Use the foollowing simple test question and first apply the `generate_sql_openai` function and then use the output for the `run_sql` function.

`"How many customers do we have?"`

In [29]:
generate_sql_openai("How many customers do we have?", SCHEMA_STR,"gpt-4o-mini")

'SELECT COUNT(*) AS total_customers FROM customers;'

Now test the following question and observe qhether it correctly links `customers` and `orders` to produce the count per customer.

```python
question = "Show the number of orders placed by each customer."

In [30]:
generate_sql_openai("show the number of orders placed by each customer", SCHEMA_STR,"gpt-4o-mini")

'SELECT c.id AS customer_id, c.name AS customer_name, COUNT(o.id) AS order_count FROM customers c LEFT JOIN orders o ON c.id = o.customer_id GROUP BY c.id, c.name LIMIT 100;'

## 🖼️ Step 4 — Formatting Results for the LLM Answer

The `run_sql` function returns a raw **Pandas DataFrame**, which is great for computation but not ideal for direct text input to another Large Language Model (LLM). For the **Answer Generation** stage, we need a clean, structured text representation of the results.

The `preview_rows_for_prompt` function is designed to convert the DataFrame into a concise, LLM-friendly **Markdown table** string.

In [36]:

def preview_rows_for_prompt(df: pd.DataFrame, max_rows: int = 30, max_cols: int = 12) -> str:
    """Turn a (small) slice of the DataFrame into a compact markdown table for the LLM prompt."""
    if df is None or df.empty:
        return "(no rows)"
    # Trim super-wide tables
    df_small = df.copy()
    if df_small.shape[1] > max_cols:
        df_small = df_small.iloc[:, :max_cols]
    # Show at most N rows
    df_small = df_small.head(max_rows)
    # Convert to nice markdown table without index
    return df_small.to_markdown(index=False)

In [ ]:
# Assuming df or  df_2 from the previous step is still available
# question = "Show the number of orders placed by each customer."

# TODO (students): Call the function on df_2 or df_2 and print the resulting string.

question = "Show the number of orders placed by each customer."
sql_query = generate_sql_openai(question, SCHEMA_STR, "gpt-4o-mini")

# SQL gegen DB ausführen und DataFrame erhalten
df_2 = run_sql(engine, sql_query)

# Ergebnis anzeigen
print(preview_rows_for_prompt(df_2, 30, 12))

|   customer_id | customer_name   |   order_count |
|--------------:|:----------------|--------------:|
|             1 | Alice           |             2 |
|             2 | Bob             |             1 |
|             3 | Choi            |             1 |
|             4 | Dara            |             0 |


## 🗣️ Step 5 — Constructing the Answer Prompt (Grounding)

After successfully executing the SQL query, the final step in the Text-to-SQL pipeline is using a second LLM call to translate the raw data back into a natural language answer.

This step is called **Answer Grounding** because the LLM is explicitly instructed to base its response **only** on the injected facts (the DataFrame rows).

In [40]:
def build_answer_prompt(question: str, sql: str, df: pd.DataFrame) -> list[dict]:
    rows_md = preview_rows_for_prompt(df)
    ANSWER_SYSTEM = """You are a helpful data assistant.
                        Given a user's question, the SQL that was executed (read-only), and the resulting rows,
                        write a concise factual answer in natural language.

                        Rules:
                        - Only use the values that actually appear in the provided rows.
                        - If the result looks incomplete or ambiguous, say so briefly and suggest a clarifying follow-up.
                        - Prefer bullet points or a short paragraph. Keep it tight.
                        - If there are numbers, include them exactly as shown (do not round unless asked).
                    """
    user_content = f"""Question:
                        {question}

                        SQL:
                        {sql}

                        Rows (first few shown):
                        {rows_md}
                    """
    return [
        {"role": "system", "content": ANSWER_SYSTEM},
        {"role": "user", "content": user_content},
    ]

# 📝 Step 6 — Generating the Final Natural Language Answer

This is the final step in the Text-to-SQL pipeline. The `llm_answer_openai` function takes the prompt generated in **Step 5** (which includes the question, SQL, and data rows) and sends it to the LLM to get the final, human-readable answer.

In [34]:
def llm_answer_openai(question: str, sql: str, df: pd.DataFrame, model: str = None) -> str:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    model = model or os.getenv("OPENAI_MODEL", "gpt-4o-mini")
    msgs = build_answer_prompt(question, sql, df)
    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=msgs,
    )
    return resp.choices[0].message.content.strip()

In [54]:
# Test the pipeline
question_3 = "which customers has ordered the most items?"
sql_query_3 = generate_sql_openai(question_3, SCHEMA_STR, "gpt-4o-mini")

# Check
#print(sql_query_3)

# SQL gegen DB ausführen und DataFrame erhalten
df_3 = run_sql(engine, sql_query_3)

# Ergebnis anzeigen
print(llm_answer_openai(question_3, sql_query_3, df_3, "gpt-4o-mini"))


def text_to_sql_qa(question, schema, engine, model="gpt-4o-mini"):
    sql = generate_sql_openai(question, schema, model)
    df = run_sql(engine, sql)
    answer = llm_answer_openai(question, sql, df, model)
    return answer

print(text_to_sql_qa("Which product name is the most popular in terms of quantity sold, and how many units were sold and total price?", SCHEMA_STR, engine, "gpt-4o-mini"))


The customers who have ordered the most items are:

- **Alice**: 4 items
- **Bob**: 2 items
- **Choi**: 1 item

If you need more details or additional customers, please let me know!
- The most popular product in terms of quantity sold is the **Monitor 27"**.
- Total units sold: **2**
- Total price: **700**


---

## Next step: move snippets into a FastAPI app

We'll extract the key functions from this notebook into a small app structure:
```
text_to_sql_app/
  ├─ app/
  │   ├─ main.py          # FastAPI routes
  │   ├─ db.py            # SQLite connection helpers
  │   ├─ prompting.py     # prompt templates & LLM call
  │   └─ safety.py        # SQL validation & guards
  ├─ tests/
  │   └─ test_safety.py
  ├─ requirements.txt
  └─ README.md
```

You can run:
```bash
uvicorn app.main:app --reload
```

We'll generate a scaffold alongside this notebook so you can iterate quickly.

### Student Task: Modularize the Code

Your task is to manually populate these files, ensuring you handle the imports correctly. All dependencies (like the `engine` or `SCHEMA_STR`) should be imported from `db.py` into `prompting.py` or passed as arguments to the functions where possible.

#### Example: `app/main.py` Scaffold

The API is already ready for use.


## 🧱 `app/database.py` — Database layer

**Purpose**
This module handles all interaction with the SQLite database: connecting, inspecting schema, and running read-only queries.

**Why it matters**
- Encapsulates database logic cleanly away from the app logic.
- Provides schema context for grounding LLM prompts.

---

### 🪜 Step breakdown

**1️⃣ Database setup** *(provided)*  
- Creates a read-only SQLAlchemy engine using `demo.db`.  
- Uses `mode=ro` to protect against accidental writes.

**2️⃣ Schema extraction** *(students implement)*  
- Use `inspect(engine)` to list tables and columns.  
- Build a compact schema string and assign it to `SCHEMA_STR`.

**3️⃣ Query execution** *(students implement)*  
- take the function `run_sql(engine, sql)` from this notebook


## 🤖 `app/openai_utils.py` — LLM interaction layer

**Purpose**
This module contains all logic that interacts with OpenAI’s API and coordinates the
Text→SQL→Answer pipeline. It connects natural language questions to grounded SQL results. the functions mentioned below are already in your notebook above.

---

### 🪜 Step breakdown

**1️⃣ Data formatting** *(students implement)*
- Implement `preview_rows_for_prompt(df, max_rows=30, max_cols=12)`  
- Implement `build_answer_prompt(question, sql, df)`  
  
---

**2️⃣ SQL generation** *(students implement)*
- Implement `generate_sql_openai(question, schema, model="gpt-4o-mini")`

---

**3️⃣ Answer generation** *(students implement)*
- Implement `llm_answer_openai(question, sql, df, model="gpt-4o-mini")`
  - Uses `build_answer_prompt()` to provide the question, executed SQL, and preview of rows.
 
---

**4️⃣ Full pipeline** *(students implement last)*
- Implement `text_to_sql_qa(question, schema, engine, model="gpt-4o-mini")`
  - Step 1: call `generate_sql_openai()`  
  - Step 2: execute SQL via `run_sql(engine, sql)`  
  - Step 3: pass results to `llm_answer_openai()`  


## 🧩 `init_db.sql` — Initialize the demo database

**Purpose**
This file contains all the SQL statements needed to create and populate the demo database.
It’s the same schema and data you use in the notebook, so simpply move the query into the file,
so it can be re-run automatically when starting the app or container.

## ▶️ Running the App Locally

We already included a **minimal Streamlit frontend** so you can interact with your Text-to-SQL service through a web UI.

To start everything locally, open **two terminal windows** (or two separate VS Code terminals) in your project root and run:

```bash
# Terminal 1 — Start the FastAPI backend
uvicorn app.main:app --reload

# Terminal 2 — Start the Streamlit frontend
streamlit run app_frontend.py